<a href="https://colab.research.google.com/github/MichalSlowakiewicz/Deep-Neural-Network/blob/master/DNN-Lab-1-MSLE-student-version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/MichalSlowakiewicz/Deep-Neural-Network/blob/master/DNN-Lab-1-MSLE-student-version.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression

In this exercise, you will use linear regression to predict flat (apartment) prices. Training will be handled via gradient descent. We will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start by obtaining the data.

In [1]:
!wget --no-verbose -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget --no-verbose -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1
!head mieszkania.csv mieszkania_test.csv

2025-10-09 18:32:37 URL:https://uc63d580ea1adee4164b2827938f.dl.dropboxusercontent.com/cd/0/inline/Cy6BmwNp7v168HMmexfS_a2B9mlYxNdFYbGKXVmid3UOAkQZ33C391W-wGcWECY1lwuSZbjeDym3LujpREtKFhb5Oqm1f1K9D_gmcKQMpkQWTRYFyEH2BWLQW6nkebgbUtw/file?dl=1 [6211/6211] -> "mieszkania.csv" [1]
2025-10-09 18:32:38 URL:https://ucabebc712e0765b3f168312f234.dl.dropboxusercontent.com/cd/0/inline/Cy4uH2Btx5bzPmTSqPfCaJMywrbLh4dKO55HXPELjYu0MZ1yJYj7rF0BFxJj7cOpRlSC3asg_f2N3PC_4o9zfLCh6r1gGaHjnbF6jj-QYtfzmgKMsDgZNmyItOKrZYQ2UYY/file?dl=1 [6247/6247] -> "mieszkania_test.csv" [1]
==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_

Each row in the data represents a separate flat. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a flat's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,...`).

We should use only `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

In [2]:
%matplotlib inline

from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from tqdm.auto import tqdm

NDArray = np.ndarray[Any, Any]

np.set_printoptions(precision=4, suppress=True)
np.random.seed(357)

## Loading and converting data

Let's start by loading the data and showing the range of prices we're working with.

In [3]:
def load(path: str) -> tuple[NDArray, NDArray]:
    """
    Returns (x, y) where:
    - x: input features, shape (n_apartments, n_features)
    - y: price, shape (n_apartments,)
    """
    data = pd.read_csv(path)
    y = data["cena"].to_numpy()
    x = data.loc[:, data.columns != "cena"].to_numpy()
    return x, y

In [4]:
x_train, y_train = load("mieszkania.csv")
x_test, y_test = load("mieszkania_test.csv")

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(200, 6) (200,)
(200, 6) (200,)


In [5]:
print(np.min(y_train), np.max(y_train), np.mean(y_train))

102572 1102309 507919.49


In [6]:
x_train[:3]

array([[104, 'mokotowo', 2, 2, 1940, 1],
       [43, 'ochotowo', 1, 1, 1970, 1],
       [128, 'grodziskowo', 3, 2, 1916, 1]], dtype=object)

We'll need to convert features to floats.

In [7]:
# Convert column 1 from str to (ordinal) int.
# (One-hot encoding would be better, but ordinal is OK for today.)
label_encoder = LabelEncoder()
label_encoder.fit(x_train[:, 1])
x_train[:, 1] = label_encoder.transform(x_train[:, 1])
x_test[:, 1] = label_encoder.transform(x_test[:, 1])

# Convert ints to float.
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

In [8]:
x_train[:3]

array([[ 104.,    1.,    2.,    2., 1940.,    1.],
       [  43.,    2.,    1.,    1., 1970.,    1.],
       [ 128.,    0.,    3.,    2., 1916.,    1.]])

## The loss and constant models

Our predictions should minimize the so-called *mean squared logarithmic error*:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's implement the loss function first.

In [9]:
def mse(ys: NDArray, ps: NDArray) -> np.float64:
    assert ys.shape == ps.shape
    return np.mean((ys - ps) * (ys - ps))

In [10]:
def msle(ys: NDArray, ps: NDArray) -> np.float64:
    assert ys.shape == ps.shape
    #################################
    # TODO: Implement this function #
    #################################
    return np.mean((np.log(1+ys)-np.log(1+ps))**2)

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [11]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################
y_mean = np.mean(y_train)
msle_error =  msle(y_test, y_mean*np.ones(len(y_test)))
print(msle_error)

0.4284115392580848


Recall that outputing the mean minimizes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [12]:
#############################################
# TODO: Find this constant and compute msle #
#############################################

y_mean = np.mean(np.log(y_train))
y_mean = np.exp(y_mean)
msle_error =  msle(y_test, np.ones(len(y_test))*y_mean)
print(msle_error)


0.42757528293624447


## Linear regression (standard)

Now, let's implement training of a standard linear regression model via gradient descent.

In [ ]:
from sklearn.linear_model import LinearRegression

def train(
    x: NDArray, y: NDArray, alpha: float = 1e-7, n_iterations: int = 100000
) -> tuple[NDArray, np.float64]:
    """Linear regression (which optimizes MSE). Returns (weights, bias)."""

    # B is batch size (number of observations).
    # F is number of (input) features.
    B, F = x.shape
    assert y.shape == (B,)

    # TODO #
    model = LinearRegression()
    model.fit(x, y)
    weights = model.coef_
    bias = model.intercept_
    return weights, bias

weights, bias = train(x_train, y_train)
preds_test =  x_test @ np.transpose(weights) + bias
print("test MSLE:", msle(y_test, preds_test))

(200, 6) (6,)
[ 5830.5496 21434.9351 12283.9731 10320.3431   670.9021 32189.9915]
test MSLE: 0.07098684418438012


In [27]:
# with gradient descendant

def train(
    x: NDArray, y: NDArray, alpha: float = 1e-7, n_iterations: int = 100000
) -> tuple[NDArray, np.float64]:
    """Linear regression (which optimizes MSE). Returns (weights, bias)."""

    # B is batch size (number of observations).
    # F is number of (input) features.
    B, F = x.shape
    assert y.shape == (B,)

    weights = np.zeros(F)
    bias = 0.0

    for i in range(n_iterations):
      y_pred = x @ weights + bias

      error = y_pred - y
      grad_w = (2/B) * (x.T @ error)
      grad_b = (2/B) * np.sum(error)

      weights -= alpha * grad_w
      bias -= alpha * grad_b

    return weights, bias

weights, bias = train(x_train, y_train)
preds_test =  x_test @ np.transpose(weights) + bias
print("test MSLE:", msle(y_test, preds_test))

test MSLE: 0.018652692632779255


## Linear regression (MSLE)

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
Use the chain rule and previous computations to get formulas for the gradient.
</sup></sub></sup></sub></sup></sub>

In [21]:
def train_msle(
    x: NDArray, y: NDArray, alpha: float = 1e-7, n_iterations: int = 100000
) -> tuple[NDArray, np.float64]:
    """Linear regression (which optimizes MSE). Returns (weights, bias)."""

    # B is batch size (number of observations).
    # F is number of (input) features.
    B, F = x.shape
    assert y.shape == (B,)

    weights = np.zeros(F)
    bias = 0.0

    for i in range(n_iterations):
      y_pred = x @ weights + bias

      log_diff = np.log1p(y) - np.log1p(y_pred)


      dL_dy_pred = -2 * log_diff / (1 + y_pred)


      grad_w = (1 / B) * (x.T @ dL_dy_pred)
      grad_b = (1 / B) * np.sum(dL_dy_pred)

      weights -= alpha * grad_w
      bias -= alpha * grad_b

    return weights, bias

weights, bias = train_msle(x_train, y_train)
preds_test =  x_test @ np.transpose(weights) + bias
print("test MSLE:", msle(y_test, preds_test))

test MSLE: 38.14321485039422


## Feature engineering

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Note that you may need to change the learning rate substantially.

Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [24]:
###############################################
# TODO: Implement the feature engineering part #
###############################################

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Load data
train = pd.read_csv("mieszkania.csv")
test = pd.read_csv("mieszkania_test.csv")

# Separate target
y_train = train["cena"].values
train = train.drop(columns=["cena"])
y_test = test["cena"].values
test = test.drop(columns=["cena"])

# One-hot encode categorical variable
enc = OneHotEncoder(sparse_output=False, drop="first")
districts_train = enc.fit_transform(train[["dzielnica"]])
districts_test = enc.transform(test[["dzielnica"]])

district_names = enc.get_feature_names_out(["dzielnica"])

# Drop original column and add encoded
train_num = train.drop(columns=["dzielnica"])
test_num = test.drop(columns=["dzielnica"])

X_train = np.concatenate([train_num.values, districts_train], axis=1)
X_test = np.concatenate([test_num.values, districts_test], axis=1)

# Convert back to DataFrame for easier manipulation
columns = list(train_num.columns) + list(district_names)
X_train = pd.DataFrame(X_train, columns=columns)
X_test = pd.DataFrame(X_test, columns=columns)

###############################################
# ✅ FEATURE ENGINEERING SECTION
###############################################

# Interaction: area * district dummies
for d in district_names:
    X_train[f"m2_{d}"] = X_train["m2"] * X_train[d]
    X_test[f"m2_{d}"] = X_test["m2"] * X_test[d]

# Other interactions
X_train["m2_sypialnie"] = X_train["m2"] * X_train["ilość_sypialni"]
X_train["m2_parking"] = X_train["m2"] * X_train["parking_podziemny"]

X_test["m2_sypialnie"] = X_test["m2"] * X_test["ilość_sypialni"]
X_test["m2_parking"] = X_test["m2"] * X_test["parking_podziemny"]

###############################################
# Now you can feed X_train, y_train into your gradient descent
###############################################

x_train = X_train.values
x_test = X_test.values


In [29]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################

def train(
    x: NDArray, y: NDArray, alpha: float = 1e-7, n_iterations: int = 100000
) -> tuple[NDArray, np.float64]:
    """Linear regression (which optimizes MSE). Returns (weights, bias)."""

    # B is batch size (number of observations).
    # F is number of (input) features.
    B, F = x.shape
    assert y.shape == (B,)

    weights = np.zeros(F)
    bias = 0.0

    for i in range(n_iterations):
      y_pred = x @ weights + bias

      error = y_pred - y
      grad_w = (2/B) * (x.T @ error)
      grad_b = (2/B) * np.sum(error)

      weights -= alpha * grad_w
      bias -= alpha * grad_b

    return weights, bias

weights, bias = train(
    x_train,
    y_train
)

# Predykcje
preds_train = x_train @ weights + bias
preds_test = x_test @ weights + bias



# Obliczenie MSLE (Mean Squared Logarithmic Error)
def msle(y_true, y_pred):
    return np.mean((np.log1p(y_true) - np.log1p(y_pred)) ** 2)

# Ewaluacja
train_msle = msle(y_train, preds_train)
test_msle = msle(y_test, preds_test)

print(f"Train MSLE: {train_msle:.6f}")
print(f"Test  MSLE: {test_msle:.6f}")

Train MSLE: 0.008397
Test  MSLE: 0.018653


# Validation

In this exercise you will implement a validation pipeline: split the non-test set into train and validation sets and select the best model based on validation results.

So far you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, you should be able to better anticipate the test time performance and compare different models and hyperparameters on datasets they are not over-fitted to.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise MSLE.

In [30]:
x_train_val, y_train_val = x_train, y_train
x_test, y_test = x_test, y_test


def random_split(
    x: NDArray, y: NDArray, val_ratio: float = 0.2
) -> tuple[tuple[NDArray, NDArray], tuple[NDArray, NDArray]]:
    """Returns (x_train, y_train), (x_val, y_val)."""

    idxs = np.random.permutation(len(x))

    ######################################################
    # TODO: Implement the basic validation split method. #
    ######################################################
    val_size = int(len(x) * val_ratio)

    val_idxs = idxs[:val_size]
    train_idxs = idxs[val_size:]

    # Tworzymy zbiory
    x_train, y_train = x[train_idxs], y[train_idxs]
    x_val, y_val = x[val_idxs], y[val_idxs]

    return (x_train, y_train), (x_val, y_val)


(x_train, y_train), (x_val, y_val) = random_split(x_train_val, y_train_val)

len(x_train), len(x_val), len(x_test)

(160, 40, 200)

In [ ]:
#############################################################
# TODO: compare MSLE on training, validation, and test sets #
#############################################################

## Cross-validation

To make the random split validation reliable, a significant chunk of training data may be needed. To get over this problem, one may apply cross-validation.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)

Let's now implement the method. Make sure that:
* number of partitions is a parameter,
* the method is not limited to `mieszkania.csv`,
* the method is not limited to one specific model.

In [ ]:
####################################
# TODO: Implement cross-validation #
####################################
def kfold(x: NDArray, y: NDArray, n_folds: int = 5, shuffle: bool = False) -> list[float]:
    """Returns losses for each fold."""



losses = kfold(x_train_val, y_train_val, n_folds=3, shuffle=False)
print(f"k-fold loss: {np.mean(losses):.4f} +- {np.std(losses):.4f}")


## Investigating input data

Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.

In [ ]:
##############################
# TODO: Investigate the data #
##############################